# Eksperiment prompt

## Set Up Module dan Environment

In [38]:
import os
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import TypedDict, List, Optional, Literal, Annotated, Set
from langchain_google_genai import ChatGoogleGenerativeAI
import traceback

from langchain_core.tools import tool
from langgraph.graph import StateGraph, END

import operator
from copy import deepcopy
from datetime import datetime
from zoneinfo import ZoneInfo

from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    Integer,
    String,
    Text,
    Date,
    DateTime,
    ForeignKey,
    UniqueConstraint,
    func,
    Boolean,
    select,
    insert,
    and_,
)

from langchain_core.messages import (
    AnyMessage,
    HumanMessage,
    ToolMessage,
    AIMessage,
    SystemMessage,
)

In [39]:
load_dotenv()


def setup_environment():
    """Memuat semua environment variables, menyimpannya ke os.environ, dan menampilkan nilai TERMASKED."""

    def mask_value(val: str, visible_fraction: float = 0.5) -> str:
        if val is None:
            return ""
        s = str(val)
        n = len(s)
        if n <= 4:
            return "*" * n
        visible = max(1, int(n * visible_fraction))
        return s[:visible] + "*" * (n - visible)

    env_vars = [
        "GOOGLE_API_KEY",
        "LANGSMITH_API_KEY",
        "LANGSMITH_TRACING",
        "LANGSMITH_ENDPOINT",
        "LANGSMITH_PROJECT",
        "DATABASE_URL",
    ]
    for var in env_vars:
        value = os.getenv(var)
        if not value:
            raise RuntimeError(
                f"{var} not found in environment. Set it in .env or export it."
            )
        os.environ[var] = value
        print(f"{var} Terload! Value: {mask_value(value)}")


setup_environment()

DATABASE_URL = os.getenv("DATABASE_URL")
engine = create_engine(DATABASE_URL)
metadata = MetaData()

GOOGLE_API_KEY Terload! Value: AIzaSyD6a8iF0_8-Ztw********************
LANGSMITH_API_KEY Terload! Value: lsv2_pt_84b8750ca1c74b349**************************
LANGSMITH_TRACING Terload! Value: ****
LANGSMITH_ENDPOINT Terload! Value: https://api.smi****************
LANGSMITH_PROJECT Terload! Value: TiketaAgen**********
DATABASE_URL Terload! Value: postgresql+psycopg://postgres:*******************************


In [40]:
SEAT_MAP: list[list[str | None]] = [
    # A
    [
        "A1",
        "A2",
        "A3",
        "A4",
        "A5",
        "A6",
        "A7",
        "A8",
        "A9",
        None,
        "A10",
        "A11",
        "A12",
        "A13",
        "A14",
        "A15",
        "A16",
        "A17",
        "A18",
    ],
    # B
    [
        "B1",
        "B2",
        "B3",
        "B4",
        "B5",
        "B6",
        "B7",
        "B8",
        "B9",
        None,
        "B10",
        "B11",
        "B12",
        "B13",
        "B14",
        "B15",
        "B16",
        "B17",
        "B18",
    ],
    # C
    [
        "C1",
        "C2",
        "C3",
        "C4",
        "C5",
        "C6",
        "C7",
        "C8",
        "C9",
        None,
        "C10",
        "C11",
        "C12",
        "C13",
        "C14",
        "C15",
        "C16",
        "C17",
        "C18",
    ],
    # D
    [
        "D1",
        "D2",
        "D3",
        "D4",
        "D5",
        "D6",
        "D7",
        "D8",
        "D9",
        None,
        "D10",
        "D11",
        "D12",
        "D13",
        "D14",
        "D15",
        "D16",
        "D17",
        "D18",
    ],
    # E
    [
        "E1",
        "E2",
        "E3",
        "E4",
        "E5",
        "E6",
        "E7",
        "E8",
        "E9",
        None,
        "E10",
        "E11",
        "E12",
        "E13",
        "E14",
        "E15",
        "E16",
        "E17",
        "E18",
    ],
    # F
    [
        "F1",
        "F2",
        "F3",
        "F4",
        "F5",
        "F6",
        "F7",
        "F8",
        "F9",
        None,
        "F10",
        "F11",
        "F12",
        "F13",
        "F14",
        "F15",
        "F16",
        "F17",
        "F18",
    ],
    # G
    [
        "G1",
        "G2",
        "G3",
        "G4",
        "G5",
        "G6",
        "G7",
        "G8",
        "G9",
        None,
        "G10",
        "G11",
        "G12",
        "G13",
        "G14",
        "G15",
        "G16",
        "G17",
        "G18",
    ],
    # H
    [
        "H1",
        "H2",
        "H3",
        "H4",
        "H5",
        "H6",
        "H7",
        "H8",
        "H9",
        None,
        "H10",
        "H11",
        "H12",
        "H13",
        "H14",
        "H15",
        "H16",
        "H17",
        "H18",
    ],
    # I
    [
        "I1",
        "I2",
        "I3",
        "I4",
        "I5",
        "I6",
        "I7",
        "I8",
        "I9",
        None,
        "I10",
        "I11",
        "I12",
        "I13",
        "I14",
        "I15",
        "I16",
        "I17",
        "I18",
    ],
    # J
    [
        "J1",
        "J2",
        "J3",
        "J4",
        "J5",
        "J6",
        "J7",
        "J8",
        "J9",
        None,
        "J10",
        "J11",
        "J12",
        "J13",
        "J14",
        "J15",
        "J16",
        "J17",
        "J18",
    ],
    # K (kosong lorong)
    [None] * 19,
    # L (lebih sempit)
    [
        "L1",
        "L2",
        "L3",
        "L4",
        "L5",
        "L6",
        "L7",
        "L8",
        "L9",
        None,
        "L10",
        "L11",
        "L12",
        "L13",
        "L14",
        "L15",
        "L16",
        "L17",
        "L18",
    ],
    # M (lebih sempit lagi)
    [
        "M1",
        "M2",
        "M3",
        "M4",
        "M5",
        "M6",
        "M7",
        "M8",
        "M9",
        None,
        "M10",
        "M11",
        "M12",
        "M13",
        "M14",
        "M15",
        "M16",
        "M17",
        "M18",
    ],
]
ALL_VALID_SEATS = {seat for row in SEAT_MAP for seat in row if seat}

In [41]:
# Define table db ke bahasa Python pakai SQLAlchemy
genres_table = Table(
    "genres",
    metadata,
    Column("id", Integer, primary_key=True, autoincrement=True),
    Column("name", String(120), nullable=False, unique=True),
    Column("created_at", DateTime, default=func.now()),
)
movies_table = Table(
    "movies",
    metadata,
    Column("id", Integer, primary_key=True, autoincrement=True),
    Column("title", String(255), nullable=False),
    Column("description", Text),
    Column("studio_number", Integer, nullable=False, unique=True),
    Column("poster_path", String(255)),
    Column("backdrop_path", String(255)),
    Column("release_date", Date),
    Column("trailer_youtube_id", String(20)),
    Column("created_at", DateTime, default=func.now()),
)
movie_genres_table = Table(
    "movie_genres",
    metadata,
    Column("movie_id", Integer, ForeignKey("movies.id"), primary_key=True),
    Column("genre_id", Integer, ForeignKey("genres.id"), primary_key=True),
)
showtimes_table = Table(
    "showtimes",
    metadata,
    Column("id", Integer, primary_key=True, autoincrement=True),
    Column("movie_id", Integer, ForeignKey("movies.id"), nullable=False),
    Column("time", DateTime, nullable=False),
    Column("is_archived", Boolean, nullable=False, default=False),
    Column("created_at", DateTime, default=func.now()),
)
bookings_table = Table(
    "bookings",
    metadata,
    Column("id", Integer, primary_key=True, autoincrement=True),
    Column("user", String(255), nullable=False),
    Column("seat", String(10), nullable=False),
    Column("showtime_id", Integer, ForeignKey("showtimes.id"), nullable=False),
    Column("created_at", DateTime, default=func.now()),
    UniqueConstraint("showtime_id", "seat", name="uq_booking_showtime_seat"),
)

## Set up helper Zona Waktu

In [42]:
TARGET_TZ = ZoneInfo("Asia/Jakarta")
UTC_TZ = ZoneInfo("UTC")


def to_utc_range_naive(date_local_str: str) -> tuple[datetime, datetime]:
    """Mengambil string 'YYYY-MM-DD' WIB, mengembalikan rentang Naive UTC."""
    try:
        local_date = datetime.strptime(date_local_str, "%Y-%m-%d").date()
    except ValueError:
        raise ValueError("Format tanggal salah. Gunakan YYYY-MM-DD.")

    start_local_aware = datetime(
        local_date.year, local_date.month, local_date.day, 0, 0, 0, tzinfo=TARGET_TZ
    )
    end_local_aware = datetime(
        local_date.year, local_date.month, local_date.day, 23, 59, 59, tzinfo=TARGET_TZ
    )
    
    start_utc_aware = start_local_aware.astimezone(UTC_TZ)
    end_utc_aware = end_local_aware.astimezone(UTC_TZ)
    
    return start_utc_aware.replace(tzinfo=None), end_utc_aware.replace(tzinfo=None)


def from_db_utc_naive_to_local_display(utc_dt_naive: datetime) -> str:
    """Mengambil Naive UTC dari DB, mengembalikan string 'HH:MM WIB'."""
    
    utc_aware = utc_dt_naive.replace(tzinfo=UTC_TZ)
    local_aware = utc_aware.astimezone(TARGET_TZ)
    
    return local_aware.strftime("%H:%M WIB")


def get_current_local_date_str() -> str:
    """Mengembalikan string 'YYYY-MM-DD' untuk hari ini di timezone LOKAL (WIB)."""
    
    return datetime.now(TARGET_TZ).strftime("%Y-%m-%d")

## StateGraph

In [43]:
class TicketAgentState(TypedDict):
    messages: Annotated[List[AnyMessage], operator.add]

    all_movies_list: List[dict]

    # --- SLOT FORMULIR ---
    current_movie_id: Optional[int]
    current_showtime_id: Optional[int]
    selected_seats: Optional[List[str]]
    customer_name: Optional[str]

    # --- KONTEKS SESAAT UNTUK SELEKTOR ---
    context_showtimes: Optional[List[dict]]
    context_seats: Optional[List[str]]

    # --- META-DATA ---
    confirmation_data: Optional[dict]
    last_error: Optional[str]

## Tools non Records

In [44]:
@tool
def get_showtimes(movie_id: int, date_local: str) -> List[dict]:
    """
    MENGAMBIL jadwal tayang untuk 1 film pada 1 tanggal LOKAL (WIB).
    Otomatis HANYA akan menampilkan jadwal yang akan datang (di atas jam sekarang).
    'date_local' HARUS dalam format 'YYYY-MM-DD'.
    """
    print(f"    > TOOL: get_showtimes(movie_id={movie_id}, date_local='{date_local}')")

    # 1. Cek apakah tanggal yang diminta adalah HARI INI (WIB)
    is_today = False
    try:
        # Panggil helper Anda untuk mendapatkan tanggal hari ini di WIB
        today_local_str = get_current_local_date_str() # -> "2025-11-03"
        if date_local == today_local_str:
            is_today = True
            print("    > INFO: Tanggal yang diminta adalah HARI INI. Menerapkan filter waktu...")
    except Exception as e:
        # Fallback jika helper error, anggap saja bukan hari ini
        print(f"    > WARNING: Gagal membandingkan tanggal hari ini: {e}")

    # 2. Dapatkan rentang UTC untuk tanggal yang diminta (Logika Lama, sudah benar)
    try:
        start_utc, end_utc = to_utc_range_naive(date_local)
    except ValueError as e:
        print(f"    > ERROR di get_showtimes: {e}")
        return [{"error": f"Format tanggal salah: {e}. Minta format YYYY-MM-DD."}]

    # 3. Bangun daftar kondisi query secara dinamis
    conditions = [
        showtimes_table.c.movie_id == movie_id,
        showtimes_table.c.time.between(start_utc, end_utc),
        showtimes_table.c.is_archived.is_(False),
    ]

    # 4. Tambahkan filter waktu jika 'is_today'
    if is_today:
        # Dapatkan waktu SEKARANG di WIB, konversi ke UTC, jadikan naive
        # Ini adalah format yang sama dengan yang disimpan di DB Anda
        now_utc_naive = datetime.now(TARGET_TZ).astimezone(UTC_TZ).replace(tzinfo=None)
        
        # Tambahkan kondisi 'WHERE time > [waktu_sekarang_di_utc]'
        conditions.append(showtimes_table.c.time > now_utc_naive)


    # 5. Buat statement query dengan SEMUA kondisi
    #    Kita menggunakan and_(*conditions) untuk "membuka" list kondisi
    stmt = (
        select(showtimes_table.c.id, showtimes_table.c.time)
        .where(and_(*conditions))  # <-- Perubahan di sini
        .order_by(showtimes_table.c.time)
    )

    # 6. Eksekusi DB
    try:
        with engine.connect() as conn:
            results = conn.execute(stmt).fetchall()
            
            if not results:
                # Beri pesan yang sedikit lebih baik jika ini hari ini
                if is_today:
                    return [
                        {"message": "Tidak ada jadwal tayang lagi untuk sisa hari ini."}
                    ]
                return [
                    {"message": "Tidak ada jadwal ditemukan untuk tanggal tersebut."}
                ]

            # Proses hasil
            showtimes_data = [
                {
                    "showtime_id": row.id,
                    "time_display": from_db_utc_naive_to_local_display(row.time),
                }
                for row in results
            ]
            print(f"    > TOOL get_showtimes: Menemukan {len(showtimes_data)} jadwal mendatang.")
            return showtimes_data
            
    except Exception as e:
        print(f"    > ERROR DB di get_showtimes: {e}")
        return [{"error": f"Gagal mengambil jadwal dari database: {e}"}]


class SeatAvailabilityInfo(BaseModel):
    count_available: int = Field(description="Jumlah kursi yang tersedia.")
    count_booked: int = Field(description="Jumlah kursi yang sudah terisi.")
    summary_for_llm: str = Field(
        description="Ringkasan tekstual kursi (tersedia & terisi) untuk prompt LLM."
    )
    available_list: List[str] = Field(description="Daftar lengkap kursi tersedia.")
    booked_list: List[str] = Field(
        description="Daftar lengkap kursi terisi."
    )


@tool
def get_available_seats(
    showtime_id: int,
) -> SeatAvailabilityInfo:
    """
    MENGAMBIL ringkasan (tersedia & terisi) dan daftar lengkap kursi tersedia
    untuk 1 jadwal.
    """
    print(f"    > TOOL: get_available_seats(showtime_id={showtime_id})")
    stmt = select(bookings_table.c.seat).where(
        bookings_table.c.showtime_id == showtime_id
    )

    try:
        booked_seats: Set[str]
        with engine.connect() as conn:
            booked_seats = {row.seat for row in conn.execute(stmt).fetchall()}

        available = sorted(
            [seat for seat in ALL_VALID_SEATS if seat not in booked_seats]
        )
        count_available = len(available)
        count_booked = len(booked_seats)
        booked_list_sorted = sorted(list(booked_seats))

        print(
            f"    > TOOL get_available_seats: dari 216 kursi, Menemukan {count_available} tersedia, {count_booked} sudah terisi."
        )

        summary_lines = []
        summary_lines.append(f"{count_available} kursi tersedia.")

        # Tampilkan info kursi yang sudah terisi (maks 10) saja
        if count_booked == 0:
            summary_lines.append("Belum ada kursi yang terisi.")
        else:
            booked_display_limit = 10
            booked_info = f"{count_booked} kursi terisi:"
            if count_booked <= booked_display_limit:
                booked_info += f" {', '.join(booked_list_sorted)}"
            else:
                booked_info += (
                    f" {', '.join(booked_list_sorted[:booked_display_limit])}..."
                )
            summary_lines.append(booked_info)

        # Tambahkan contoh kursi tersedia jika masih relevan
        if 0 < count_available <= 20:
            summary_lines.append(f"Kursi tersedia: {', '.join(available)}")
        elif count_available > 20:
            mid_index = count_available // 2
            examples = sorted(
                list(set([available[0], available[mid_index], available[-1]]))
            )
            summary_lines.append(
                f"Contoh kursi tersedia: {examples[0]} ... {examples[-1]}"
            )

        summary_str = " ".join(summary_lines)

        # Kembalikan dalam format dictionary/Pydantic
        return SeatAvailabilityInfo(
            count_available=count_available,
            count_booked=count_booked,
            summary_for_llm=summary_str,
            available_list=available,
            booked_list=booked_list_sorted,
        )

    except Exception as e:
        print(f"    > ERROR DB di get_available_seats: {e}")
        return SeatAvailabilityInfo(
            count_available=0,
            count_booked=0,
            summary_for_llm=f"Error: Gagal mengambil data kursi: {e}",
            available_list=[],
            booked_list=[],
        )


class AskUserSchema(BaseModel):
    question: str = Field(
        description="Pertanyaan yang jelas dan spesifik untuk diajukan ke user."
    ) # Perlu diubah ini nanti karena konflik instruksi


@tool(args_schema=AskUserSchema)
def ask_user(question: str) -> str:
    """
    Gunakan tool ini untuk MENGIRIM PESAN ke user.
    Bisa untuk BERTANYA (minta input) ATAU MEMBERI INFORMASI (misal daftar jadwal/kursi)
    sebelum bertanya.
    """
    print(f"    > TOOL: ask_user(question='{question}')")
    return question


@tool
def signal_confirmation_ready():
    """
    Panggil tool ini HANYA JIKA SEMUA 4 slot (movie_id, showtime_id, seats, customer_name)
    SUDAH TERISI LENGKAP. Ini adalah sinyal bahwa formulir sudah siap.
    """
    print("    > TOOL: signal_confirmation_ready() dipanggil.")
    return "Sinyal konfirmasi diterima."


# --- TOOL MANUAL ---
def book_tickets_tool(showtime_id: int, seats: List[str], customer_name: str) -> str:
    """Fungsi Python murni untuk eksekusi booking."""
    print(
        f"    > EKSEKUSI: Mencoba booking {seats} untuk {customer_name} di showtime {showtime_id}"
    )
    insert_data = [
        {"showtime_id": showtime_id, "seat": s, "user": customer_name} for s in seats
    ]
    try:
        with engine.connect() as conn:
            with conn.begin():
                # Validasi kursi sebelum insert (Defensive)
                invalid_seats = [s for s in seats if s not in ALL_VALID_SEATS]
                if invalid_seats:
                    raise ValueError(
                        f"Kursi tidak valid ditemukan: {', '.join(invalid_seats)}"
                    )

                # Cek ketersediaan lagi (Defensive, race condition)
                stmt_check = select(bookings_table.c.seat).where(
                    and_(
                        bookings_table.c.showtime_id == showtime_id,
                        bookings_table.c.seat.in_(seats),
                    )
                )
                already_booked = conn.execute(stmt_check).fetchall()
                if already_booked:
                    booked_list = [r.seat for r in already_booked]
                    raise ValueError(
                        f"Kursi {', '.join(booked_list)} sudah terisi saat mencoba booking."
                    )

                # Insert jika aman
                conn.execute(insert(bookings_table), insert_data)

        return f"Sukses! Tiket untuk {customer_name} di kursi {', '.join(seats)} telah dikonfirmasi."
    except ValueError as ve:
        print(f"    > EKSEKUSI GAGAL (Validasi): {ve}")
        return f"Maaf, terjadi masalah: {ve}"
    except Exception as e:
        print(f"    > EKSEKUSI GAGAL (DB): {e}")
        if "uq_booking_showtime_seat" in str(e):
            return f"Maaf, terjadi error saat booking. Salah satu kursi ({', '.join(seats)}) mungkin sudah terisi oleh orang lain."
        return "Maaf, terjadi error tak terduga saat booking."


class MovieDetails(BaseModel):
    title: str
    synopsis: str
    trailer_url: Optional[str] = Field(
        default=None, description="URL YouTube lengkap jika tersedia."
    )
    error: Optional[str] = Field(default=None)

@tool
def get_movie_details(movie_id: int) -> MovieDetails:
    """
    MENGAMBIL detail (sinopsis, trailer) untuk 1 film.
    Gunakan ini JIKA user bertanya 'filmnya tentang apa'. atau 'trailer filmnya dong!'.
    Tool ini HANYA mengambil info, TIDAK mencatat pilihan film.
    """
    print(f"     > TOOL: get_movie_details(movie_id={movie_id})")
    try:
        with engine.connect() as conn:
            stmt = select(
                movies_table.c.title,
                movies_table.c.description,
                movies_table.c.trailer_youtube_id,
            ).where(movies_table.c.id == movie_id)
            result = conn.execute(stmt).first()

            if result:
                trailer_id = result.trailer_youtube_id
                full_trailer_url = None
                
                if trailer_id:
                    full_trailer_url = f"https://www.youtube.com/watch?v={trailer_id}"

                return MovieDetails(
                    title=result.title,
                    synopsis=result.description or "Sinopsis tidak tersedia.",
                    trailer_url=full_trailer_url,
                )
            else:
                return MovieDetails(error="Film tidak ditemukan.")
    except Exception as e:
        print(f"     > ERROR di get_movie_details: {e}")
        return MovieDetails(error=f"Error database: {e}")

## Testing Tools (work semua)

In [45]:
# int(get_showtimes.func(2, "2025-10-30")) #OK
# print(get_available_seats.func(718)) #O
# rint(ask_user.func("Siapa nama Anda?")) #OK
# print(get_movie_details.func(2)) #OK
# print(book_tickets_tool(718, ["F4","F5","F6","F7", "F8", "F9"], "Rafi Wangsa")) # perlu terpisah

## Schemas dan Tools Record

In [46]:
class SelectMovieAction(BaseModel):
    """
    Aksi untuk MENGISI slot 'current_movie_id'.
    Pilih ini HANYA jika kamu sudah 100% yakin ID filmnya.
    """

    selected_movie_id: int = Field(
        description="ID film yang sudah pasti (dari 'all_movies_list')."
    )


class SelectShowtimeAction(BaseModel):
    """
    Aksi untuk MENGISI slot 'current_showtime_id'.
    Pilih ini HANYA jika kamu sudah 100% yakin ID jadwalnya.
    """

    selected_showtime_id: int = Field(
        description="ID jadwal yang sudah pasti (dari 'context_showtimes')."
    )


class SelectSeatsAction(BaseModel):
    """
    Aksi untuk MENGISI slot 'selected_seats'.
    Pilih ini HANYA jika kamu sudah 100% yakin kursinya.
    """

    selected_seats_list: List[str] = Field(description="Daftar kursi yang sudah pasti. Format WAJIB: huruf kapital dan angka (contoh: 'A1', 'B5').")


class ExtractNameAction(BaseModel):
    """
    Aksi untuk MENGISI slot 'customer_name'.
    Pilih ini HANYA jika kamu sudah 100% yakin namanya.
    """

    extracted_customer_name: str = Field(description="Nama pemesan yang sudah pasti.")


@tool(args_schema=SelectMovieAction)
def record_selected_movie(selected_movie_id: Optional[int]) -> str:
    """
    Gunakan ini untuk MENCATAT ID film yang sudah dipilih user.
    Panggil ini SETELAH kamu mencocokkan input user ('Kimi no Nawa')
    ke ID film dari 'DAFTAR FILM TERSEDIA'.
    Jika user tidak memilih/tidak relevan, panggil dengan 'selected_movie_id: null'.
    """
    if selected_movie_id is None:
        return "OK. Tidak ada film yang dipilih."
    return f"OK. Film ID {selected_movie_id} dicatat."


@tool(args_schema=SelectShowtimeAction)
def record_selected_showtime(selected_showtime_id: Optional[int]) -> str:
    """
    Gunakan ini untuk MENCATAT ID jadwal yang sudah dipilih user.
    Panggil ini SETELAH kamu mencocokkan input user ('jam 7 malam')
    ke ID jadwal dari 'Jadwal Tersedia'.
    Jika user tidak memilih/tidak relevan, panggil dengan 'selected_showtime_id: null'.
    """
    if selected_showtime_id is None:
        return "OK. Tidak ada jadwal yang dipilih."
    return f"OK. Jadwal ID {selected_showtime_id} dicatat."


@tool(args_schema=SelectSeatsAction)
def record_selected_seats(selected_seats_list: Optional[List[str]]) -> str:
    """
    Gunakan ini untuk MENCATAT daftar kursi yang sudah dipilih user.
    Panggil ini SETELAH kamu mengekstrak kursi dari input user.
    CONTOH: ['A1', 'A2'].
    Format kursi WAJIB HURUF KAPITAL diikuti angka (misal: ['A1', 'B5']), sesuai PETA KURSI.
    """
    if not selected_seats_list:
        return "OK. Tidak ada kursi yang dipilih."
    return f"OK. Kursi {', '.join(selected_seats_list)} dicatat."


@tool(args_schema=ExtractNameAction)
def record_customer_name(extracted_customer_name: Optional[str]) -> str:
    """
    Gunakan ini untuk MENCATAT nama pemesan yang sudah diekstrak.
    Panggil ini SETELAH kamu mengekstrak nama (misal 'Rafi') dari input user.
    Jika user tidak menyebut nama, panggil dengan 'extracted_customer_name: null'.
    """
    if not extracted_customer_name:
        return "OK. Tidak ada nama yang diekstrak."
    return f"OK. Nama {extracted_customer_name} dicatat."

## Initisasi Model

In [47]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

## Helper Functions 

In [48]:
def get_focus_instruction(state: TicketAgentState) -> str:
    """
    Menentukan instruksi fokus yang FLEKSIBEL, memberi LLM pilihan
    antara aksi (record) atau bertanya (ask_user).
    """

    # --- SLOT 1: current_movie_id ---
    if not state.get("current_movie_id"):
        return (
            "FOKUS SAAT INI: Selesaikan 'current_movie_id'. "
            "Jika user sudah jelas memilih film (dari 'DAFTAR FILM TERSEDIA'), panggil 'record_selected_movie'. "
            "Jika belum jelas atau perlu bantuan, WAJIB panggil 'ask_user' (tampilkan daftar & bertanya). DEFAULT -> ask_user."
        )

    # --- SLOT 2: current_showtime_id ---
    if not state.get("current_showtime_id"):
        context_showtimes = state.get("context_showtimes")

        if not context_showtimes:
            return (
                "FOKUS SAAT INI: Dapatkan jadwal. "
                "User HARUS menyebut TANGGAL pemutaran. Anda BOLEH menerima frasa relatif seperti 'hari ini', "
                "'besok', atau 'lusa' (termasuk hari dalam minggu, mis. 'Senin depan'). "
                "Hitung sendiri tanggal 'YYYY-MM-DD' berdasarkan 'KONTEKS WAKTU SAAT INI'. "
                "Jika tanggal belum jelas, WAJIB panggil 'ask_user' untuk meminta tanggal (boleh relatif) (DEFAULT). "
                "Jika tanggal sudah jelas, panggil 'get_showtimes(movie_id, date_local)'."
            )

        if (
            isinstance(context_showtimes, list)
            and len(context_showtimes) > 0
            and isinstance(context_showtimes[0], dict)
            and ("error" in context_showtimes[0] or "message" in context_showtimes[0])
        ):
            return (
                "FOKUS SAAT INI: Jadwal tidak ditemukan atau error. "
                "WAJIB panggil 'ask_user' untuk memberi tahu & minta tanggal lain (DEFAULT)."
            )

        return (
            "FOKUS SAAT INI: Tentukan 'current_showtime_id'. "
            "Jika user sudah jelas memilih, panggil 'record_selected_showtime'. "
            "Jika perlu klarifikasi, WAJIB panggil 'ask_user' (DEFAULT)."
        )

    # --- SLOT 3: selected_seats ---
    if not state.get("selected_seats"):
        context_seats_summary = state.get("context_seats_summary", "N/A")

        if context_seats_summary == "N/A":
            return (
                "FOKUS SAAT INI: Dapatkan info kursi. "
                "WAJIB panggil 'get_available_seats' untuk mengambil ringkasan kursi."
            )

        if (
            "Error:" in context_seats_summary
            or "0 kursi tersedia" in context_seats_summary
        ):
            return (
                "FOKUS SAAT INI: Kursi tidak tersedia atau error. "
                "WAJIB panggil 'ask_user' untuk memberi tahu & sarankan opsi (DEFAULT)."
            )

        return (
            "FOKUS SAAT INI: Tentukan 'selected_seats'. "
            "Perhatikan PETA KURSI: Format kursi yang valid adalah huruf KAPITAL diikuti angka (misal: 'A1', 'B2'). "
            "Lakukan kognisi: Jika user mengetik 'a1' atau 'kursi b2', Anda WAJIB menafsirkannya dan memanggil 'record_selected_seats' dengan format yang benar dan tervalidasi (misal: ['A1', 'B2']). "
            "Periksa juga 'Kursi Tersedia' di PETA KURSI untuk memastikan kursi yang dipilih user belum terisi. "
            "Jika user memilih kursi yang SUDAH TERISI, WAJIB panggil 'ask_user' untuk memberi tahu (misal: 'Maaf, kursi B2 sudah terisi. Silakan pilih kursi lain.'). "
            "Jika user deskriptif/tidak pasti (misal: 'kursi depan paling tengah!'), WAJIB panggil 'ask_user' untuk saran/klarifikasi (misal: 'apakah yang kamu maksud kursi pada baris M9 atau M10?') (DEFAULT)."
        )

    # --- SLOT 4: customer_name ---
    if not state.get('customer_name'):
        return (
            "FOKUS SAAT INI: Tentukan 'customer_name'. "
            "PERIKSA HISTORI CHAT DENGAN TELITI. "
            "1. Jika user SUDAH JELAS menyebutkan namanya (misal: 'atas nama Budi', 'nama saya Budi'), panggil tool `record_customer_name(extracted_customer_name='Budi')`. "
            "2. Jika user memberi petunjuk nama TAPI ANDA TIDAK 100% YAKIN (misal: 'Budi di sini'), WAJIB panggil 'ask_user' untuk KONFIRMASI (misal: 'Untuk konfirmasi, pemesanan ini atas nama Budi?'). "
            "3. Jika TIDAK ADA NAMA SAMA SEKALI, WAJIB panggil 'ask_user' untuk MINTA NAMA (misal: 'Baik, pemesanan ini atas nama siapa?') (DEFAULT)."
        )

    return "FOKUS SAAT INI: Semua formulir sudah terisi. WAJIB panggil 'signal_confirmation_ready'."

## Master Prompt

In [ ]:
def get_simple_master_prompt(state: TicketAgentState) -> List[AnyMessage]:
    """
    Merakit System Prompt LENGKAP dengan aturan, konteks, dan pengecualian.
    """

    # --- 1. Ambil Semua Data Konteks ---
    movie_id = state.get("current_movie_id")
    showtime_id = state.get("current_showtime_id")
    seats = state.get("selected_seats")
    customer = state.get("customer_name")

    today_date_str = get_current_local_date_str()

    movie_list_str = "\n".join(
        [
            f"- ID: {m['id']}, Judul: {m['title']}"
            for m in state.get("all_movies_list", [])
        ]
    )
    if not movie_list_str:
        movie_list_str = "Error: Daftar film tidak ter-load."

    showtime_list = state.get("context_showtimes", [])
    showtime_context_str = "N/A (Panggil 'get_showtimes' dulu)"
    if showtime_list:
        if isinstance(showtime_list[0], dict) and "error" in showtime_list[0]:
            showtime_context_str = f"Error: {showtime_list[0]['error']}"
        elif isinstance(showtime_list[0], dict) and "message" in showtime_list[0]:
            showtime_context_str = showtime_list[0]["message"]
        else:
            showtime_context_str = "\n".join(
                [
                    f"- ID: {s['showtime_id']}, Waktu: {s['time_display']}"
                    for s in showtime_list
                ]
            )

    seat_context_str = state.get(
        "context_seats_summary", "N/A (Panggil 'get_available_seats' dulu)"
    )

    # --- 2. Dapatkan Instruksi Fokus ---
    focus_instruction = get_focus_instruction(state)

    # --- 3. Rakit Peta Kursi ---
    seat_map_str_lines = []
    for row_list in SEAT_MAP:
        row_str = " ".join([seat if seat else "____" for seat in row_list])
        seat_map_str_lines.append(row_str)
    seat_map_context_str = "\n".join(seat_map_str_lines)

    # --- 3. Rakit Prompt ---
    prompt_lines = [
        "Anda adalah Manajer Booking. Tugas Anda mengisi formulir.",
        "Prioritas UTAMA Anda adalah menyelesaikan `INSTRUKSI FOKUS`.",
        "Anda WAJIB memanggil TEPAT SATU tool yang paling relevan dengan fokus tersebut.",
        "\nKEWAJIBAN OUTPUT (PENTING):",
        "- Selalu kembalikan respons dalam BENTUK tool_calls. Dilarang keras menjawab teks biasa tanpa tool.",
        "- Jika ingin bertanya/menyampaikan informasi ke user, WAJIB gunakan tool `ask_user(question: str)`.",
        "- Jika ragu atau tidak ada tool lain yang pasti, DEFAULT-kan ke `ask_user`.",
        "- Jika Anda tidak memanggil tool, sistem akan menganggapnya error.",
        "\nCHECKLIST PEMILIHAN TOOL (DEFAULT -> ask_user):",
        "- Perlu klarifikasi/bertanya/menyajikan daftar ke user -> ask_user.",
        "- Sudah punya tanggal -> get_showtimes; belum punya tanggal -> ask_user (minta tanggal).",
        "- Sudah punya showtime_id dan perlu ketersediaan -> get_available_seats; ingin menyarankan/bertanya kursi -> ask_user.",
        "- Semua slot sudah terisi -> signal_confirmation_ready.",
        "\nCONTOH SALAH (JANGAN):",
        "Assistant: Baik, Anda memilih kursi B2 dan C4. Atas nama siapa pemesanan ini?",
        "CONTOH BENAR (WAJIB tool):",
        'ask_user(question="Baik, Anda memilih kursi B2 dan C4. Atas nama siapa pemesanan ini?")',
        "\n**ATURAN PENGECUALIAN (MUNDUR):**",
        "Aturan ini MENGALAHKAN `INSTRUKSI FOKUS`:",
        "Jika pesan user terbaru **jelas-jelas** ingin MENGGANTI slot yang SUDAH TERISI (misal: 'ganti film', 'ganti jadwal'),",
        "ABAIKAN FOKUS UTAMA dan WAJIB panggil tool `record_...` yang sesuai (misal: `record_selected_movie`) untuk menimpa data lama.",
        "\n**ATURAN PENGECUALIAN (INFO FILM):**",
        "Aturan ini juga MENGALAHKAN `INSTRUKSI FOKUS`:",
        "Jika user bertanya tentang detail film ('tentang apa', 'sinopsis', 'trailer'),",
        "1. ABAIKAN FOKUS UTAMA (misal: jangan minta tanggal).",
        "2. Cocokkan nama film (jika perlu) ke 'DAFTAR FILM TERSEDIA' untuk dapat ID-nya.",
        "3. Panggil tool `get_movie_details(movie_id)`.",
        "4. **PENTING: JANGAN** panggil `record_selected_movie` (karena user hanya bertanya, belum tentu memilih).",
        "\n**ATURAN PENGECUALIAN (JADWAL UMUM):**",
        "Aturan ini juga MENGALAHKAN `INSTRUKSI FOKUS`:",
        "Jika user bertanya pertanyaan umum tentang hari tayang (misal: 'film X tayang hari apa saja?', 'mainnya kapan aja?'),",
        "1. ABAIKAN FOKUS UTAMA (misal: jangan minta film/kursi).",
        "2. JANGAN panggil `get_showtimes` (karena tool itu butuh tanggal spesifik).",
        "3. WAJIB panggil `ask_user` dengan jawaban ini: 'Film di sini tayang setiap hari, namun pemesanan tiket hanya bisa dilakukan untuk hari ini, besok, dan lusa. Anda ingin memesan untuk tanggal berapa? Hari ini atau besok?'",        
        "\nDILARANG KERAS menjawab langsung. Jika bingung, panggil 'ask_user'.",
        "\n**KONTEKS WAKTU SAAT INI:**",
        f"Hari ini (WIB) adalah tanggal: **{today_date_str}**",
        "Gunakan tanggal ini sebagai referensi WAJIB Anda.",
        "Jika user bilang 'hari ini', gunakan tanggal ini.",
        "Jika user bilang 'besok', 'lusa', atau 'minggu depan', Anda WAJIB menghitung tanggal 'YYYY-MM-DD' yang benar berdasarkan tanggal hari ini.",
        f"\n**DAFTAR FILM TERSEDIA:**\n{movie_list_str}",
        f"\n**FORMULIR SAAT INI:**\n"
        f"- current_movie_id: {movie_id or 'BELUM ADA'}\n"
        f"- current_showtime_id: {showtime_id or 'BELUM ADA'}\n"
        f"- selected_seats: {seats or 'BELUM ADA'}\n"
        f"- customer_name: {customer or 'BELUM ADA'}",
        f"\n**KONTEKS TAMBAHAN:**\n"
        f"- Jadwal Tersedia:\n{showtime_context_str}\n"
        f"- Kursi Tersedia: {seat_context_str}\n"
        f"- Error Terakhir: {state.get('last_error') or 'Tidak ada'}",
        f"\n**PETA KURSI (SEAT_MAP):**\n{seat_map_context_str}",
        f"\n**INSTRUKSI FOKUS:**\n{focus_instruction}",
    ]

    system_prompt_content = "\n".join(prompt_lines)
    return [SystemMessage(content=system_prompt_content)]

## Function Manage History

In [50]:
def get_stable_history_slice(
    messages: List[AnyMessage], max_messages: int = 32
) -> List[AnyMessage]:
    """Ambil potongan histori stabil tanpa memutus pasangan AI/tool."""
    if not messages:
        return []

    buffer = max_messages + 6
    sliced = messages[-buffer:]

    # Jangan mulai dengan ToolMessage
    while sliced and isinstance(sliced[0], ToolMessage):
        sliced.pop(0)

    # Jangan mulai dengan AIMessage yang berisi tool_calls (model turn harus didahului user/tool)
    while sliced and isinstance(sliced[0], AIMessage) and sliced[0].tool_calls:
        sliced.pop(0)

    # Batasi ukuran
    sliced = sliced[-max_messages:]

    # Bersihkan lagi awal
    while sliced and isinstance(sliced[0], ToolMessage):
        sliced.pop(0)
    while sliced and isinstance(sliced[0], AIMessage) and sliced[0].tool_calls:
        sliced.pop(0)

    # PENTING: Jangan AKHIRI slice dengan AIMessage yang berisi tool_calls
    # (Gemini mengharuskan function call diikuti function response sebelum model dipanggil lagi)
    while sliced and isinstance(sliced[-1], AIMessage) and sliced[-1].tool_calls:
        sliced.pop()

    return sliced

## Booking Manager

In [51]:
booking_manager_tools = [
    get_showtimes,
    get_available_seats,
    ask_user,
    get_movie_details,
    signal_confirmation_ready,
    record_selected_movie,
    record_selected_showtime,
    record_selected_seats,
    record_customer_name,
]

model_with_tools = llm.bind_tools(booking_manager_tools)


def node_booking_manager(state: TicketAgentState) -> dict:
    print("--- NODE: Booking Manager ---")

    # 1. Dapatkan HANYA system prompt
    system_prompt_list = get_simple_master_prompt(state)

    # 2. GABUNGKAN system prompt dengan histori chat dari state
    safe_history = get_stable_history_slice(state.get("messages", []), max_messages=32)
    messages_for_llm = system_prompt_list + safe_history

    # Siapkan dictionary untuk update state
    current_summary = state.get("context_seats_summary", "N/A")
    updates = {
        "messages": [],
        "last_error": None, # Sementara gini dulu aja, penanganan error masih agak problem
        "context_seats_summary": current_summary, # Biar konteks kursi tetap terbawa
    }
    ai_response = None

    try:
        # 3. SELALU Panggil LLM untuk Tool Call
        print("     > Meminta Tool Call...")
        ai_response = model_with_tools.invoke(messages_for_llm)
        print(f"     > Hasil LLM (Tool Call): {ai_response.tool_calls}")
        updates["messages"].append(ai_response)

        if not ai_response.tool_calls:
            print(
                "     > Peringatan: LLM tidak memanggil tool (atau mungkin memang tidak perlu)."
            )
            updates["last_error"] = "LLM gagal memanggil tool."

        # 4. Proses SEMUA tool call
        for tool_call in ai_response.tool_calls:
            tool_name = tool_call["name"]
            tool_args = tool_call["args"]
            tool_id = tool_call["id"]

            tool_result_content = "Aksi dicatat."  # Default untuk tool aksi

            if tool_name == "record_selected_movie":
                selected_id = tool_args.get("selected_movie_id")
                if selected_id is not None:
                    updates["current_movie_id"] = selected_id
                    # Reset konteks bawahan
                    updates["context_showtimes"] = None
                    updates["current_showtime_id"] = None
                    updates["context_seats"] = None
                    updates["selected_seats"] = None
                    updates["context_seats_summary"] = "N/A"

            elif tool_name == "record_selected_showtime":
                selected_id = tool_args.get("selected_showtime_id")
                if selected_id is not None:
                    updates["current_showtime_id"] = selected_id
                    updates["context_seats"] = None
                    updates["selected_seats"] = None
                    updates["context_seats_summary"] = "N/A"

            elif tool_name == "record_selected_seats":
                seats_list = tool_args.get("selected_seats_list")
                if seats_list:
                    updates["selected_seats"] = (
                        seats_list
                    )
                else:
                    updates["last_error"] = "Agent mencoba rekam kursi kosong."
                    tool_result_content = "Error: Daftar kursi kosong."

            elif tool_name == "record_customer_name":
                name = tool_args.get("extracted_customer_name")
                if name:
                    updates["customer_name"] = name


            elif tool_name == "get_showtimes":
                try:
                    # EKSEKUSI tool-nya SEKARANG
                    showtimes_result = get_showtimes.invoke(tool_args)
                    updates["context_showtimes"] = showtimes_result  # Simpan konteks
                    tool_result_content = str(showtimes_result)
                except Exception as e:
                    updates["last_error"] = f"Gagal fetch showtimes: {e}"
                    tool_result_content = f"Error: {e}"
                    updates["context_showtimes"] = [
                        {"error": str(e)}
                    ]

            elif tool_name == "get_available_seats":
                try:
                    # EKSEKUSI tool-nya SEKARANG
                    seat_info: SeatAvailabilityInfo = get_available_seats.invoke(
                        tool_args
                    )
                    updates["context_seats"] = seat_info.available_list
                    updates["context_seats_summary"] = seat_info.summary_for_llm
                    tool_result_content = str(seat_info.model_dump())
                except Exception as e:
                    updates["last_error"] = f"Gagal fetch seats: {e}"
                    updates["context_seats_summary"] = "Error ambil kursi."
                    tool_result_content = f"Error: {e}"

            elif tool_name == "get_movie_details":
                # Eksekusi tool dan kirim hasilnya ke LLM sebagai ToolMessage
                try:
                    details: MovieDetails = get_movie_details.invoke(tool_args)
                    # Kirim payload lengkap agar LLM bisa menyusun jawaban/ask_user berikutnya
                    tool_result_content = str(details.model_dump())
                except Exception as e:
                    updates["last_error"] = f"Gagal fetch movie details: {e}"
                    tool_result_content = f"Error: {e}"

            elif tool_name == "ask_user":
                # EKSEKUSI tool-nya SEKARANG
                tool_result_content = ask_user.invoke(tool_args)
                # Router akan menangani __end__

            elif tool_name == "signal_confirmation_ready":
                # EKSEKUSI tool-nya SEKARANG
                tool_result_content = signal_confirmation_ready.invoke(tool_args)
                # Router akan menangani 'confirm'

            updates["messages"].append(
                ToolMessage(content=tool_result_content, tool_call_id=tool_id)
            )

    except Exception as e:
        print(f"     > ERROR saat pemanggilan LLM: {e}")
        updates["last_error"] = f"Gagal memproses langkah: {e}"
        if ai_response is None:
            error_msg = AIMessage(content=f"Maaf, terjadi error internal: {e}")
            updates["messages"].append(error_msg)

    return updates

## Node Confirm

In [52]:
def node_confirmation(state: TicketAgentState) -> dict:
    """
    Mengambil data DARI STATE, menampilkan rangkuman, dan mengeksekusi booking.
    Dipicu HANYA setelah tool 'signal_confirmation_ready' dipanggil.
    """
    print("--- NODE: Confirmation ---")

    movie_id = state.get("current_movie_id")
    showtime_id = state.get("current_showtime_id")
    seats = state.get("selected_seats")
    customer_name = state.get("customer_name")

    final_data = {
        "movie_id": movie_id,
        "showtime_id": showtime_id,
        "seats": seats,
        "customer_name": customer_name,
    }

    if not all(final_data.values()):  # Cek jika salah satu masih None
        print(
            f"    > ERROR: Data konfirmasi tidak lengkap di state! Data: {final_data}"
        )
        return {
            "messages": [
                AIMessage(
                    content=f"Terjadi error: Data pemesanan tidak lengkap untuk konfirmasi. Data: {final_data}"
                )
            ],
            "last_error": "Data tidak lengkap saat konfirmasi.",
        }

    # 2. Ambil detail (Nama film, Waktu tampil) untuk rangkuman
    # (Ini butuh query kecil ke DB)
    movie_title = "(Judul tidak ditemukan)"
    showtime_display = "(Jadwal tidak ditemukan)"
    try:
        with engine.connect() as conn:
            # Ambil judul film
            movie_res = conn.execute(
                select(movies_table.c.title).where(
                    movies_table.c.id == final_data["movie_id"]
                )
            ).first()
            if movie_res:
                movie_title = movie_res.title

            # Ambil waktu jadwal (dan konversi ke WIB display)
            showtime_res = conn.execute(
                select(showtimes_table.c.time).where(
                    showtimes_table.c.id == final_data["showtime_id"]
                )
            ).first()
            if showtime_res:
                showtime_display = from_db_utc_naive_to_local_display(showtime_res.time)

    except Exception as e:
        print(f"    > ERROR saat mengambil detail untuk konfirmasi: {e}")

    # 3. Buat Rangkuman Teks
    summary = (
        f"✅ **Konfirmasi Pesanan Anda:**\n"
        f"---------------------------\n"
        f"🎬 **Film:** {movie_title} (ID: {final_data['movie_id']})\n"
        f"🗓️ **Jadwal:** {showtime_display} (ID: {final_data['showtime_id']})\n"
        f"💺 **Kursi:** {', '.join(final_data['seats'])}\n"
        f"👤 **Atas Nama:** {final_data['customer_name']}\n"
        f"---------------------------\n"
        f"\n⏳ Memproses pemesanan..."
    )
    # Tampilkan rangkuman ke konsol (opsional)
    print(f"    > Rangkuman:\n{summary}")

    # 4. Eksekusi Booking (Panggil fungsi Python biasa)
    result_message = book_tickets_tool(
        showtime_id=final_data["showtime_id"],
        seats=final_data["seats"],
        customer_name=final_data["customer_name"],
    )

    print(f"    > Hasil Eksekusi: {result_message}")

    # 5. Kembalikan Pesan Final ke User
    booking_url = f"https://tiketa-rafi.space/book/{final_data['showtime_id']}"
    final_response = (
        f"{summary}\n\n"
        f"**Status:** {result_message}\n\n"
        f"🔗 Buka halaman showtime: {booking_url}\n"
        f"[Klik di sini untuk membuka]({booking_url})"
    )

    # Reset state setelah booking
    updates = {
        "messages": [AIMessage(content=final_response)],
        "current_movie_id": None,
        "current_showtime_id": None,
        "selected_seats": None,
        # "customer_name": None,
        "context_showtimes": None,
        "context_seats": None,
        "confirmation_data": None,
        "last_error": None,
    }
    return updates

## Booking Router


In [53]:
def booking_router(
    state: TicketAgentState,
) -> Literal["confirm", "continue", "__end__"]:
    print("--- ROUTER: Booking Router ---")

    if state.get("last_error"):
        print(f"    > Rute: __end__ (ERROR terdeteksi: {state.get('last_error')})")
        return "__end__"

    messages = state["messages"]
    last_ai_message = None
    for msg in reversed(messages):
        if isinstance(msg, AIMessage):
            last_ai_message = msg
            break

    if not last_ai_message or not last_ai_message.tool_calls:
        print("    > Rute: __end__ (NO_TOOL_CALL / LLM bandel)")
        return "__end__"

    # Ambil tool call *pertama* (atau terakhir, tergantung logikamu)
    tool_call = last_ai_message.tool_calls[0]
    tool_name = tool_call["name"]

    if tool_name == "signal_confirmation_ready":
        print("    > Rute: confirm (data lengkap)")
        return "confirm"
    elif tool_name == "ask_user":
        print("    > Rute: __end__ (menunggu input user)")
        return "__end__"
    else:
        # Ini adalah tool Aksi (record_...) ATAU tool Data (get_...)
        print(f"    > Rute: continue (Aksi/Tool '{tool_name}' dipanggil, lanjut loop)")
        return "continue"

## Workflow Graph

In [54]:
print("Merakit Graph...")
workflow = StateGraph(TicketAgentState)

workflow.add_node("booking_manager", node_booking_manager)
workflow.add_node("confirmation", node_confirmation)

workflow.set_entry_point("booking_manager")

workflow.add_conditional_edges(
    "booking_manager",  # asal
    booking_router,
    {
        "confirm": "confirmation",
        "continue": "booking_manager",  # Looping
        "__end__": END,  # Jika router bilang '__end__', graph berhenti
    },
)

# Untuk node konfirmasi selalu END
workflow.add_edge("confirmation", END)

app = workflow.compile()
print("Graph berhasil di-compile.")


def get_all_movies_from_db():
    print("Memuat daftar film dari DB...")
    try:
        with engine.connect() as conn:
            if "title" not in movies_table.c:
                raise KeyError("Kolom 'title' tidak ditemukan di movies_table.")
            rows = conn.execute(
                select(movies_table.c.id, movies_table.c.title)
            ).fetchall()
            if not rows:
                print("PERINGATAN: Tidak ada film ditemukan di database.")
            return [{"id": row.id, "title": row.title} for row in rows]
    except Exception as e:
        print(f"ERROR saat memuat daftar film: {e}")
        print("Menggunakan daftar film contoh sebagai fallback.")
        return [
            {"id": 1, "title": "Spirited Away"},
            {"id": 2, "title": "Your Name"},
            {"id": 3, "title": "Attack on Titan: Requiem"},
        ]


ALL_MOVIES_CONTEXT = get_all_movies_from_db()
print(f"Total {len(ALL_MOVIES_CONTEXT)} film dimuat ke konteks.")


INITIAL_STATE: TicketAgentState = {
    "messages": [],
    "all_movies_list": ALL_MOVIES_CONTEXT,
    "current_movie_id": None,
    "current_showtime_id": None,
    "selected_seats": None,
    "customer_name": None,
    "context_showtimes": None,
    "context_seats": None,
    "context_seats_summary": "N/A",
    "confirmation_data": None,
    "last_error": None,
}

# PENYIMPANAN STATE (PERLU DIIMPLEMENTASIKAN LEBIH NANTI)
session_states = {}
SESSION_ID = "user_session_123"

print("\n--- Agen Manajer Booking Siap! ---")
print("Ketik 'exit' untuk keluar.")
print("Contoh: 'mau pesan tiket', 'kimi no nawa', '2025-10-30', 'A1', 'Rafi'")

Merakit Graph...
Graph berhasil di-compile.
Memuat daftar film dari DB...
Total 21 film dimuat ke konteks.

--- Agen Manajer Booking Siap! ---
Ketik 'exit' untuk keluar.
Contoh: 'mau pesan tiket', 'kimi no nawa', '2025-10-30', 'A1', 'Rafi'


## Run

In [ ]:
# ...existing code...

## Run - Gradio Chat UI
try:
    import gradio as gr
except ImportError:
    import sys, subprocess
    # Install otomatis jika belum ada (opsional, bisa dihapus jika sudah install via terminal)
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "gradio>=4.44.0"])
    import gradio as gr

# Catatan:
# - Tetap menggunakan INITIAL_STATE, session_states, SESSION_ID yang sudah didefinisikan di atas agar kompatibel.
# - Tetap mencetak input user dan output agen ke console, plus semua print log dari tools/nodes Anda akan tampil di output notebook/console.

def _compute_output_to_show(final_state) -> str:
    # Reuse logika penentuan output dari loop sebelumnya
    agent_messages = final_state.get("messages", [])
    last_agent_message = agent_messages[-1] if agent_messages else None

    output_to_show = "(Agen tidak memberikan respons)"

    if isinstance(last_agent_message, AIMessage):
        if getattr(last_agent_message, "tool_calls", None):
            # Cek apakah tool call terakhir adalah ask_user
            last_tool_call = last_agent_message.tool_calls[-1]
            if last_tool_call["name"] == "ask_user":
                output_to_show = last_tool_call["args"]["question"]
            else:
                # Untuk tool lain, tampilkan pesan generik
                output_to_show = "(Agen sedang memproses...)"
        elif getattr(last_agent_message, "content", None):
            # Tampilkan content jika tidak ada tool call (jawaban/konfirmasi final)
            output_to_show = last_agent_message.content
    elif isinstance(last_agent_message, ToolMessage):
        # Jika berakhir di ToolMessage (fallback)
        output_to_show = last_agent_message.content

    return output_to_show


def process_message(user_text: str, chat_history: list, state_dict: dict):
    # Pastikan ada state
    if not state_dict:
        state_dict = deepcopy(INITIAL_STATE)

    # Tampilkan input user di console
    print(f"\nAnda: {user_text}")

    # Tambahkan pesan user ke state
    state_dict["messages"].append(HumanMessage(content=user_text))

    # Tanda di console bahwa agen mulai
    print("\nAgen:")

    # Panggil graph
    final_state = app.invoke(state_dict, {"recursion_limit": 50})
    if final_state is None:
        bot_text = "Error: Graph tidak menghasilkan state akhir."
        print(bot_text)
        # Tampilkan di UI chat
        chat_history.append((user_text, bot_text))
        return chat_history, state_dict

    # Simpan state sesi (kompatibel dengan pola sebelumnya)
    session_states[SESSION_ID] = final_state

    # Tentukan output yang ditampilkan ke user (kompatibel dengan logika lama)
    bot_text = _compute_output_to_show(final_state)

    # Tampilkan output agen di console
    print(bot_text)

    # Tambahkan ke UI chat
    chat_history.append((user_text, bot_text))
    return chat_history, final_state


def reset_state():
    # Reset state sesi dan bersihkan riwayat chat
    session_states[SESSION_ID] = deepcopy(INITIAL_STATE)
    print("\n[RESET] State direset.")
    return [], session_states[SESSION_ID]


with gr.Blocks(title="Agen Manajer Booking") as demo:
    gr.Markdown("### Agen Manajer Booking\nKetik pesan di bawah untuk memulai.")
    chatbot = gr.Chatbot(label="Percakapan", height=420)  # list[tuple[user, bot]]
    with gr.Row():
        txt = gr.Textbox(placeholder="Tulis pesan Anda...", scale=4)
        send = gr.Button("Kirim", variant="primary", scale=1)
        clear = gr.Button("Reset", variant="secondary")

    # State gradio untuk membawa state graph antar pesan
    state_gr = gr.State(value=deepcopy(INITIAL_STATE))

    def on_submit(user_text, history, state):
        if not user_text or not user_text.strip():
            # Tidak kirim apa-apa jika kosong
            return history, state, gr.update(value="")
        history, state = process_message(user_text.strip(), history or [], state)
        # Kosongkan textbox setelah kirim
        return history, state, gr.update(value="")

    # Klik tombol Kirim
    send.click(on_submit, inputs=[txt, chatbot, state_gr], outputs=[chatbot, state_gr, txt])
    # Tekan Enter di textbox
    txt.submit(on_submit, inputs=[txt, chatbot, state_gr], outputs=[chatbot, state_gr, txt])

    def on_clear():
        history, state = reset_state()
        return history, state

    # Klik tombol Reset
    clear.click(on_clear, outputs=[chatbot, state_gr])

# Jalankan UI di notebook; jika di script, akan buka di http://127.0.0.1:7860
demo.launch(inline=True, show_error=True)

C:\Users\RafiWangsaSeniang\AppData\Local\Temp\ipykernel_2888\1185508636.py:88: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Percakapan", height=420)  # list[tuple[user, bot]]


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.



Anda: kimi no nawa adanya jadwal hari ini apa aja?

Agen:
--- NODE: Booking Manager ---
     > Meminta Tool Call...
     > Hasil LLM (Tool Call): [{'name': 'get_showtimes', 'args': {'date_local': '2025-11-03', 'movie_id': 2}, 'id': '34f51526-6d13-48ed-84d3-da34c37e7b4e', 'type': 'tool_call'}]
    > TOOL: get_showtimes(movie_id=2, date_local='2025-11-03')
    > INFO: Tanggal yang diminta adalah HARI INI. Menerapkan filter waktu...
    > TOOL get_showtimes: Menemukan 3 jadwal mendatang.
--- ROUTER: Booking Router ---
    > Rute: continue (Aksi/Tool 'get_showtimes' dipanggil, lanjut loop)
--- NODE: Booking Manager ---
     > Meminta Tool Call...
     > Hasil LLM (Tool Call): [{'name': 'record_selected_movie', 'args': {'selected_movie_id': 2}, 'id': 'b961dc13-a8a5-4ad8-9cb9-f95a2e10ff96', 'type': 'tool_call'}]
--- ROUTER: Booking Router ---
    > Rute: continue (Aksi/Tool 'record_selected_movie' dipanggil, lanjut loop)
--- NODE: Booking Manager ---
     > Meminta Tool Call...
     > Hasil

# Notes